# Anyscale Endpoints with AIConfig

[Anyscale Endpoints](https://www.anyscale.com/endpoints) support optimized inference for many open source models, including the LLaMA2 family of models (7B, 13B, 70B, CodeLLaMA, LLaMA Guard) and Mistral (7B, Mixtral 8x7B).

This cookbook shows how to use any [Anyscale Endpoints](https://www.anyscale.com/endpoints) model with AIConfig using the same simple API.

In this notebook we show how to build a travel itinerary using inference from multiple models (LLaMA-7B and Mixtral-8x7B) using Anyscale Endpoints

Read more about [AIConfig for prompt and model management](https://github.com/lastmile-ai/aiconfig).

## Models supported with Anyscale Endpoints
For the complete list, please see https://app.endpoints.anyscale.com/docs
* LLaMA-7B: meta-llama/Llama-2-7b-chat-hf
* LLaMA-13B: meta-llama/Llama-2-13b-chat-hf
* LLaMA-70B: meta-llama/Llama-2-70b-chat-hf
* LLaMA Guard: Meta-Llama/Llama-Guard-7b
* CodeLLaMA: codellama/CodeLlama-34b-Instruct-hf
* Mistral-7B (OpenOrca): Open-Orca/Mistral-7B-OpenOrca
* Mistral-7B: mistralai/Mistral-7B-Instruct-v0.1
* Mixtral-8x7B: mistralai/Mixtral-8x7B-Instruct-v0.1
* Zephyr: HuggingFaceH4/zephyr-7b-beta
* GTE: thenlper/gte-large

# Installation and Setup

In [1]:
#%pip install -e /Users/saqadri/lm/aiconfig/python
%pip install python-aiconfig

Obtaining file:///Users/saqadri/lm/aiconfig/python
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for python-aiconfig (pyproject.toml) ... done
  Created wheel for python-aiconfig: filename=python_aiconfig-1.1.8-0.editable-py3-none-any.whl size=9971 sha256=fbca0ed6d950008e262466231d7c55fab845a22af39eb637e26a284722253ad7
  Stored in directory: /private/var/folders/mk/5l10lyqs1c73_pj2grj9f9vw0000gn/T/pip-ephem-wheel-cache-1j_z2ajl/wheels/e7/fd/a2/adfd3d754fa5bcf2040054afef13bbf40a8e091249b821d62d
Successfully built python-aiconfig
  Attempting uninstall: python-aiconfig
    Found existing installation: python-aiconfig 1.1.8
    Uninstalling python-aiconfig-1.1.8:
      Successfully uninstalled python-aiconfig-1.1.8
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Create .env file containing the following line:
# ANYSCALE_ENDPOINT_API_KEY=<your key here>
# You can get your key from https://app.endpoints.anyscale.com/docs (https://app.endpoints.anyscale.com/credentials)

import dotenv
dotenv.load_dotenv()

## Load AI Config `travel.aiconfig.json`
Download file [from here](https://github.com/lastmile-ai/aiconfig/blob/main/cookbooks/Anyscale/travel.aiconfig.json). If you're using Colab, upload to 'Files' in your colab notebook and load (shown below).


In [2]:
from aiconfig import AIConfigRuntime, InferenceOptions, CallbackManager

# Load the aiconfig.
config = AIConfigRuntime.load('travel.aiconfig.json')
config.callback_manager = CallbackManager([])

/opt/homebrew/Caskroom/miniconda/base/envs/aiconfig/lib/python3.12/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/aiconfig/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Run `get_activities` prompt with streaming (llama-7b)

In [3]:
# Run a single prompt (with streaming). inference_options = InferenceOptions(stream=True)

inference_options = InferenceOptions(stream=True)
activities = await config.run("get_activities", options=inference_options)

  Sure! Here are 10 fun attractions to do in New York City:

1. Take a walk across the Brooklyn Bridge: This iconic bridge offers stunning views of the Manhattan skyline, the East River, and the Brooklyn shore. It's a must-do activity for anyone visiting NYC.
2. Explore the Metropolitan Museum of Art: With over 2 million works of art spanning 5,000 years of human history, the Met is one of the world's greatest museums. Be sure to spend at least a whole day here to see everything.
3. Visit Central Park: This 843-acre green oasis in the middle of Manhattan offers a peaceful escape from the hustle and bustle of the city. Take a stroll through the park, rent a bike, or have a picnic on the Great Lawn.
4. Go to a Broadway show: NYC is home to some of the world's most iconic theaters and productions. Catch a musical or play on Broadway for an unforgettable experience.
5. Visit the 9/11 Memorial & Museum: This moving tribute to the victims of the 9/11 attacks features two large reflecting poo

# Run `gen_itinerary` prompt chain (mixtral-8x7b)

**Note**: In `travel.aiconfig.json`, `gen_itinerary` is defined as:

```json
{
    "name": "gen_itinerary",
    "input": "Generate an itinerary ordered by {{order_by}} for these activities: {{get_activities.output}}.",
    "metadata": {
        "model": {
            "name": "AnyscaleEndpoint",
            "settings": {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "max_tokens": 3000,
            "system_prompt": "You are an expert travel coordinator with exquisite taste."
            }
        },
        "parameters": {
            "order_by": "geographic location"
        }
    }
}
```

This means it depends on the output of the `get_activities` prompt, and uses that to run inference with mixtral-8x7B. Let's run this prompt chain with AIConfig

In [4]:
# This time, let's also add a callback manager to log more detailed events
config.callback_manager = CallbackManager.create_default_manager()

itinerary = await config.run(
    "gen_itinerary",
    params={"order_by": "duration"},
    run_with_dependencies=True,
    options=inference_options)

  Sure! Here are 10 fun attractions to do in New York City:

1. Visit the Statue of Liberty and Ellis Island: Take a ferry to Liberty Island to see the iconic Statue of Liberty up close and learn about its history at the nearby Ellis Island Immigration Museum.
2. Explore the Metropolitan Museum of Art: With over 2 million works of art spanning 5,000 years of history, the Met is one of the world's largest and most comprehensive art museums.
3. Walk across the Brooklyn Bridge: Take a stroll across this iconic bridge for spectacular views of the Manhattan skyline, the East River, and Brooklyn.
4. See a Broadway show: New York City is home to dozens of world-class theaters and performances, including Tony-award winning shows like Hamilton, Wicked, and The Lion King.
5. Visit Central Park: This 843-acre green oasis in the middle of Manhattan offers a peaceful escape from the hustle and bustle of the city, with plenty of walking paths, lakes, and monuments to explore.
6. Go shopping on Fifth

# Add prompt to config

In [5]:
# Add gen_packing_list as a prompt to config
model = "gpt-3.5-turbo"
data = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You provide a bulleted list of items to pack for a week long trip."
        },
        {
            "role": "user",
            "content": "What should I bring to {{location}}?"
        }
    ]
}

new_prompts = await config.serialize(model, data, prompt_name="gen_packing_list", params={"location": "nyc"})
config.add_prompt(prompt_name="gen_packing_list", prompt_data=new_prompts[0])

In [7]:
packing_list = await config.run("gen_packing_list", params=None, options=inference_options)

- Clothes for a week, including underwear, socks, and sleepwear
- Outerwear appropriate for the weather (jacket, coat, or sweater)
- Comfortable shoes for walking
- Toiletries (toothbrush, toothpaste, shampoo, conditioner, soap, etc.)
- Medications, if needed
- Phone charger and other electronic accessories
- Personal identification and travel documents (ID, passport, visa, etc.)
- Wallet with cash and/or credit cards
- Travel-sized umbrella or raincoat, depending on the weather forecast
- Snacks and water bottle for the journey
- Any necessary electronics or gadgets (camera, laptop, etc.)
- Entertainment options (books, magazines, headphones, etc.)
- Travel adapter, if necessary
- Maps or guidebook of NYC
- Sunscreen and sunglasses, if visiting during summer or sunny weather
- Light backpack or day bag for touring around the city
- Any special items specific to your needs or interests (art supplies, workout gear, etc.)

## Pause and appreciate what we just did

We ran the `get_activities` prompt with LLaMA-7B, the `gen_itinerary` prompt (which depends on `get_activities`) with Mixtral-8x7B, and now just added a `gen_packing_list` prompt with gpt-3.5. 

Anyscale helps us get optimized inference for open source models, and AIConfig enables us to mix and match different model providers (OpenAI, Anyscale and others) in a model-agnostic generative AI application.

# 5. Save the AIConfig


In [8]:
# Save the aiconfig to disk. and serialize outputs from the model run
config.save('updated.aiconfig.json', include_outputs=True)

### Save the aiconfig as a YAML file

AIConfig supports both JSON and YAML modes, allowing you to save a config representing the generative AI signature of your AI application in the format of your choice.

In [10]:
config.save('travel.aiconfig.yaml', include_outputs=False, mode="yaml")

# 6. Open the AIConfig in AI Workbook Playground
1. Download `updated.aiconfig.json`.
2. Go to https://lastmileai.dev.
3. Go to Workbooks page: https://lastmileai.dev/workbooks.
4. Click dropdown '+New Workbook' and select 'Create from AIConfig'
5. Upload `updated.aiconfig.json`.

Try out the workbook playground here: [NYC Travel Workbook](https://lastmileai.dev/workbooks/clooqs3p200kkpe53u6n2rhr9)